# Read PDF information with LLMs
### let's test how good it the LLMs at extracting information from pdfs

In [2]:
packages = ['pymongo', 'gridfs', 'python-dotenv', 'openai']
for package in packages:
    try:
        __import__(package)
    except ImportError:
        !pip install {package}

import os
from pymongo import MongoClient
from gridfs import GridFS
from dotenv import load_dotenv
from openai import OpenAI

#endpoint = '10.0.10.5:27017' # use vpn
endpoint = '192.168.1.10:27017'
#endpoint = '192.168.229.55:27017'
database = 'supercias'
collection = 'companies'
pdfs_path = '../storage/pdfs/'
# make dir path if it does not exist
if not os.path.exists(pdfs_path):
    os.makedirs(pdfs_path)

# Connect to MongoDB
db = MongoClient('mongodb://'+endpoint)[database]
collection = db[collection]

#find pdf file
comp = collection.find_one()
print(comp['name'])


docs = comp['Documentos online']['downloaded']
pdf_names = []
#if company has documents
for value in docs.values():
    if isinstance(value, list) and len(value) > 0:
        [ pdf_names.append(v) for v in value ]


Defaulting to user installation because normal site-packages is not writeable
GISIS S.A.


Download an pdf file to make sure that it is working

In [42]:
pdf_filename = pdf_names[20]

def write_to_disk(filename):
    print(f'fetching {filename} to disk...')
    # get pdf from GridFS
    fs = GridFS(db, collection='companies')
    pdf  = fs.find({'filename': filename})
    try:
        pdf = next(pdf)
    except StopIteration:
        #throw error
        raise FileNotFoundError(f'File {filename} not found in GridFS')


    # write file to disk
    with open(pdfs_path+filename, 'wb') as f:
        f.write(pdf.read())
        return f.name

def delete_from_disk(filename):
    print(f'deleting {filename} from disk...')
    try:
        os.remove(pdfs_path+filename)
        return True
    except FileNotFoundError:
        raise FileNotFoundError(f'File {filename} not found in disk')

write_to_disk(pdf_filename)

fetching 0991295437001_DocumentosGenerales_Oficio de Presentación de Doc. por el Adm  RL  RL Sub_2012-02-08_BISCAYNE HARBOR ASSETS LLP.pdf to disk...


'../storage/pdfs/0991295437001_DocumentosGenerales_Oficio de Presentación de Doc. por el Adm  RL  RL Sub_2012-02-08_BISCAYNE HARBOR ASSETS LLP.pdf'

Make sure that the the openai api is workig correcly

In [23]:
# Load the .env file
load_dotenv()
# Example usage
api_key = os.getenv('OPENAI_KEY')
# Initialize the OpenAI client
client = OpenAI(
    api_key=api_key,
)

def prompt_openai(prompt):
    # Create a chat completion
    chat_completion = client.chat.completions.create(
        messages=[ prompt ],
        model="gpt-3.5-turbo",
    )
    return chat_completion.choices[0].message.content

print(prompt_openai({ "role": "system", "content": "is this working correcly?" }))

Yes, it appears to be working correctly. How can I assist you today?


uploading the pdf dirrecly to openai is not possible. it is not able to extract the data as of today (2024-09-02). 

So we need to extract the text first.

In [37]:
import fitz  # PyMuPDF
import pytesseract
from pdf2image import convert_from_path
from PIL import Image
import io

def extract_text_from_image_pdf(pdf_filename):
    # Open the PDF file
    pdf_document = fitz.open( pdfs_path+pdf_filename )
    text = ""

    for page_num in range(pdf_document.page_count):
        page = pdf_document[page_num]
        
        # Extract images from the page
        image_list = page.get_images(full=True)
        
        # If there are images, use OCR to extract text
        if image_list:
            for image_index, img in enumerate(page.get_images(full=True)):
                xref = img[0]
                base_image = pdf_document.extract_image(xref)
                image_bytes = base_image["image"]

                # Convert to a PIL Image
                image = Image.open(io.BytesIO(image_bytes))
                
                # Use pytesseract to do OCR on the image
                text += pytesseract.image_to_string(image)
        else:
            # If no images, extract text as usual
            text += page.get_text()

    return text

extracted_text = extract_text_from_image_pdf(pdf_filename)
print(extracted_text)

rs) =z
1.- FORMULARIO DE NOMINA DE SOCIOS O ACCIONISTAS DE UNA Compania

SENORES SUPERINTENDENCIA DE COMPANIAS DE GUAYAQUIL

hm
o>
Sone

ad

Dando cumplimiento a lo que dispone la Ley Reformatoria a la ley d@compaittas

publicada en el Registro Oficial 591 del 15 de Mayo del 2009 y las resollKiones Nas.
siguiente documentacion:

SC.SG.G.09.02 y SC.SG.G.09.03 del Superintendente de Compaiiias, gdiunto “A

EXTRANJERA QUE A SU VEZ ES SOCIA O ACCIONISTA DE COMPAN
ECUATORIANA, debidamente firmado por el representante legal de la

compafiia BISCAYNE HARBOR ASSETS LLP, tanto del periodo 2010-2011, como
el del periodo 2011-2012.

FORMULARIO DE NOMINA DE SOCIOS O ACCIONISTAS DE UNA COMPANIA
EXTRANJERA QUE A SU VEZ ES SOCIA O ACCIONISTA DE COMPANIA
ECUATORIANA, debidamente firmado por el representante legal de la

companiia NETVENTURES GLOBAL INC, tanto del periodo 2010 a Julio de 2011 y
Agosto de 2011 a la actualidad (2.012)

Copia notariada del CERTIFICADO DE EXISTENCIA LEGAL Y DE DIGNATARIOS


Now ask the summerize correcly the infomation extracted from the extracted text

In [27]:
prompt = "Give me all the information you can aquire from this text. return it in json file " + pdf_filename + ": \n"+  extracted_text
response = prompt_openai({ "role": "system", 
    "content": prompt,
})
print(response)

{
  "document_title": "Oficio de Presentación de Documentos por el Administrador RL RL Sub 2012-02-08 BISCAYNE HARBOR ASSETS LLP.pdf",
  "information_presented": [
    {
      "topic": "FORMULARIO DE NOMINA DE SOCIOS O ACCIONISTAS DE UNA Compania",
      "details": "Se presenta el formulario de nómina de socios o accionistas de una compañía extranjera que a su vez es socia o accionista de una compañía ecuatoriana, firmado por el representante legal de la empresa BISCAYNE HARBOR ASSETS LLP para los periodos 2010-2011 y 2011-2012."
    },
    {
      "topic": "FORMULARIO DE NOMINA DE SOCIOS O ACCIONISTAS DE UNA COMPANIA EXTRANJERA",
      "details": "Se presenta el formulario de nómina de socios o accionistas de una compañía extranjera que a su vez es socia o accionista de una compañía ecuatoriana, firmado por el representante legal de la empresa NETVENTURES GLOBAL INC para los periodos 2010 a julio de 2011 y agosto de 2011 a la actualidad (2012)."
    },
    {
      "topic": "Certificad

now let iterate trhough every document to see if we can get a infomration fo the company as a whole

In [44]:
analysis = db['analysis']
# for every pdf file in company
for pdf in pdf_names:
    try:
        # read the file
        write_to_disk(pdf)
        print(f'processing {pdf}...')
        # extract text
        text = extract_text_from_image_pdf(pdf)
        if(len(text) > 16385):
            # too big for openai
            print('text too big for openai')
            continue
        # intepret the data with the llm
        response = prompt_openai({
            "role": "system", 
            "content": "Give me all the information you can aquire from this text. give it in json file " + pdf + ": \n" + text, 
        })
        print('response:', response)
        # save the data
        analysis.insert_one({ 'filename': pdf, 'text_extracted': text, 'interpretation': response })
    # catch any errors
    except Exception as e:
        print(f'Error processing {pdf}: {e}')




fetching 0991295437001_DocumentosGenerales_Oficio Transferencia Acciones_1994-09-13.pdf to disk...
Error processing 0991295437001_DocumentosGenerales_Oficio Transferencia Acciones_1994-09-13.pdf: File 0991295437001_DocumentosGenerales_Oficio Transferencia Acciones_1994-09-13.pdf not found in GridFS
fetching 0991295437001_DocumentosGenerales_Oficio Transferencia Acciones_1996-09-06.pdf to disk...
Error processing 0991295437001_DocumentosGenerales_Oficio Transferencia Acciones_1996-09-06.pdf: File 0991295437001_DocumentosGenerales_Oficio Transferencia Acciones_1996-09-06.pdf not found in GridFS
fetching 0991295437001_DocumentosGenerales_Oficio Transferencia Acciones_2008-05-08.pdf to disk...
Error processing 0991295437001_DocumentosGenerales_Oficio Transferencia Acciones_2008-05-08.pdf: File 0991295437001_DocumentosGenerales_Oficio Transferencia Acciones_2008-05-08.pdf not found in GridFS
fetching 0991295437001_DocumentosGenerales_Oficio Transferencia Acciones_2009-01-14.pdf to disk...
E